In [5]:
# import Inception V3 model and combine with dense layers
import keras
from keras import models
from keras import layers
from keras.applications import inception_v3

conv_base = keras.applications.inception_v3.InceptionV3(include_top=False, 
                         weights='imagenet', 
                         input_shape=(640,480,3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 18, 13, 2048)      21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 479232)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               122683648 
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 144,486,689
Trainable params: 144,452,257
Non-trainable params: 34,432
_________________________________________________________________


In [9]:
# freeze the weights of conv layers

print('This is the number of trainable weights before freezing the conv base: ', len(model.trainable_weights))
conv_base.trainable = False
print('This is the number of trainable weights after freezing the conv base: ', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base:  192
This is the number of trainable weights after freezing the conv base:  4


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_dir=''
validation_dir=''

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(640,480),
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(640,480),
    batch_size=20,
    class_mode='binary')

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=2e-5),
             metrics=['acc'])

In [ ]:
# Activate the model training process, and eventually save the model

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50)

model.save('OM_Check_InceptionV3_2.h5')

In [ ]:
# 顯示訓練和驗證週期的損失值和準確度曲線

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Predict with Saved Model

In [1]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

test_dir = ''

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 18 ,13, 2048))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(directory,
                                           target_size=(640,480),
                                           batch_size=batch_sizem
                                           class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch) # 從批次影像中萃取特徵
        features[i * batch_size : (i+1) * batch_size] = features_batch
        labels[i * batch_size : (i+1) * batch_size] = labels_batch
        i += 1
        print(i, end=' ') # 檢視特徵萃取進度
        if i * batch_size >= sample_count: #當萃取樣本數超過設定的樣本數時，執行break中斷
            break
    return features, labels

test_features, test_labels = extract_features(test_dir, 45) # Numbers of test samples

test features = np.reshape(test_features, (45, 18*13*2048))

predict_result = model.predict_classes(test_features)

print()

correct_num = 0

for i in range(len(test_features)):
    print(str(predict_result[i]), ', Correct Answer is 'm str(test_labels[i]))
    if predict_result[i] == test_labels[i]:
        correct_num = correct_num + 1
        
print('Accuracy: ', correct_num/len(test_features))